# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of filepaths to csv files in event_data directory
file_path_list = [item.path for item in os.scandir(filepath) if item.is_file()]

print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# print(len(full_data_rows_list))
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)


#### Set Keyspace

In [7]:
session.set_keyspace('sparkify')


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  
## Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Write a query
select1 = """
SELECT artist, song, length FROM songplays 
WHERE
    session_id = 338 and
    item_in_session = 4
"""

# Create table relevant to query
create1 = "CREATE TABLE IF NOT EXISTS songplays (session_id int, item_in_session int, artist text, song text, length decimal, PRIMARY KEY ((session_id, item_in_session)))"
session.execute(create1)
       
# Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
                
    for line in csvreader:
        query = "INSERT INTO songplays (session_id, item_in_session, artist, song, length) "
        query = query + "VALUES (%s, %s ,%s, %s, %s)"
        # Casting to appropriate types as reading csv rows returns list of strings
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))) 

                
## View data to verify if it was entered into the table and provide answer to the query
rows = session.execute(select1)
for row in rows:
    print(*row)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [9]:
## Query 2: 
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Write a query
select2 = """
SELECT artist, song, first_name, last_name FROM user_activity
WHERE
    user_id = 10 and
    session_id = 182
"""

# Create table relevant to query
create2 = """
CREATE TABLE IF NOT EXISTS user_activity
(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text,
    PRIMARY KEY ((user_id, session_id), item_in_session))
"""
session.execute(create2)

# Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_activity (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s ,%s, %s, %s, %s, %s)"
        # Casting to appropriate types as reading csv rows returns list of strings
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        
## View data to verify if it was entered into the table and provide answer to the query
rows = session.execute(select2)
for row in rows:
    print(*row)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [10]:
## Query 3:
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Write a query
select3 = """
SELECT first_name, last_name FROM users_by_song
WHERE
    song = 'All Hands Against His Own'
"""

# Create table relevant to query
create3 = """
CREATE TABLE IF NOT EXISTS users_by_song
(song text, user_id int, first_name text, last_name text,
    PRIMARY KEY (song, user_id))
"""
session.execute(create3)

# Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s ,%s, %s)"
        # Casting to appropriate types as reading csv rows returns list of strings
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute(select3)
for row in rows:
    print(*row)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [11]:
drop1 = "DROP TABLE IF EXISTS songplays"
session.execute(drop1)

drop2 = "DROP TABLE IF EXISTS user_activity"
session.execute(drop2)

drop3 = "DROP TABLE IF EXISTS users_by_song"
session.execute(drop3)

### Close the session and cluster connection¶

In [12]:
session.shutdown()
cluster.shutdown()

## OPTIONAL: Question for the reviewer
 
If you have any question about the starter code or your own implementation, please add it in the cell below. 

For example, if you want to know why a piece of code is written the way it is, or its function, or alternative ways of implementing the same functionality, or if you want to get feedback on a specific part of your code or get feedback on things you tried but did not work.

Please keep your questions succinct and clear to help the reviewer answer them satisfactorily. 

> Dear reviewer,
>
> This was a good exercise. My only concern is related to how the data is loaded. We pick certain columns from the partitioned files only to create a new file which we iterate over yet again. My doubt is that this seems not too optimal. 
>
> Moreover I had encountered problems with that approach. It worked first time right but at a second time the os.walk  has also captured .ipynb_checkpoints directory in the file_path_list. I used os.scandir instead as all files are in a single directory and we do not need to include any subdirectories.
>
> Faithfully, Jakub Pitera